In [1]:
import requests
from bs4 import BeautifulSoup
from xml.etree import ElementTree
import pandas as pd
import time
import numpy as np
import json
import geocoder

### get data from the Zillow

In [20]:
def GetDataFromWeb(num):
    count = 0
    listingsFromWeb = []
    for i in range(1,num): # 21
        page_url = "http://www.zillow.com/homes/for_sale/Boston-MA/44269_rid/any_days/42.4379,-70.629044,42.191135,-71.310883_rect/10_zm/%d_p/" %i
        page = requests.get(page_url).text
        page_soup = BeautifulSoup(page, 'html.parser')
        item = page_soup.find("div", {"id": 'search-results' }).find_all("div", {"class": "zsg-photo-card-caption"})
        if not len(item) == 26:
            print i, len(item)
        else:
            listingsFromWeb += (item)
        time.sleep(2)
        count += 1
        print i
    return listingsFromWeb

listingsFromWeb = GetDataFromWeb(28)

1


In [21]:
print len(listingsFromWeb)
print len(listingsFromWeb[0])

print listingsFromWeb[0]
print listingsFromWeb[1]
print listingsFromWeb[2]

26
4
<div class="zsg-photo-card-caption"><h4 class="zsg-photo-card-spec"><span class="zsg-photo-card-status"><span class="zsg-icon-for-sale"></span>House For Sale</span></h4><p class="zsg-photo-card-spec"><span class="zsg-photo-card-price">$899,000</span><span class="zsg-photo-card-info">5 bds <span class="interpunct">·</span> 2 ba <span class="interpunct">·</span> 2,423 sqft</span></p><p class="zsg-photo-card-spec"><span class="zsg-photo-card-notification ">8 days on Zillow</span><span class="interpunct">·</span><span class="zsg-photo-card-address">54 Grampian Way, Dorchester, MA</span></p><p class="zsg-photo-card-spec"><span class="zsg-photo-card-broker-name">Gibson Sotheby's International Realty</span></p></div>
<div class="zsg-photo-card-caption"><h4 class="zsg-photo-card-spec"><span class="zsg-photo-card-status"><span class="zsg-icon-for-sale"></span>Condo For Sale</span></h4><p class="zsg-photo-card-spec"><span class="zsg-photo-card-price">$529,000</span><span class="zsg-photo-ca

### dictionary from the data for future data manipulation

In [22]:
data = []
# print BeautifulSoup(listings2)
# soup = BeautifulSoup(listings2[0])
for i in range(len(listingsFromWeb)):#http://localhost:8888/notebooks/hosingPrice/scraping_property_price.ipynb#
    theDict = {}
#     theDict['price'] = listingsFromWeb[i].find("zsg-photo-card-price")
    theDict['status'] = listingsFromWeb[i].find("span", { "class" : "zsg-photo-card-status" }).text
    theDict['address'] = listingsFromWeb[i].find("span", { "class" : "zsg-photo-card-address" }).text
    # print listings2[0].find("span", { "class" : "zsg-photo-card-broker-name" }).text
    try:
        theDict['prices'] = listingsFromWeb[i].find("span", { "class" : "zsg-photo-card-price" }).text.split("$")[1]
    except:
        theDict['prices'] = -1;
    try:
        theJson = json.loads(listingsFromWeb[i].find("script", { "type" : "application/ld+json" }).text)
        theDict['lat'] = theJson["location"]["geo"]["latitude"]
        theDict['long'] = theJson["location"]["geo"]["longitude"]
    except:
        theDict['lat'] = -1
        theDict['long'] = -1
    data.append(theDict)



In [23]:
print len(data)
for i in data:
    print i

26
{'status': u'House For Sale', 'prices': u'899,000', 'lat': -1, 'long': -1, 'address': u'54 Grampian Way, Dorchester, MA'}
{'status': u'Condo For Sale', 'prices': u'529,000', 'lat': -1, 'long': -1, 'address': u'7-9 Herbertson # 2, Jamaica Plain, MA'}
{'status': u'Condo For Sale', 'prices': u'849,000', 'lat': 42.33145, 'long': -71.05343, 'address': u'321 Dorchester St # 1, Boston, MA'}
{'status': u'House For Sale', 'prices': u'739,900', 'lat': -1, 'long': -1, 'address': u'31 Maple St, Boston, MA'}
{'status': u'House For Sale', 'prices': u'849,000', 'lat': 42.277522, 'long': -71.171084, 'address': u'129 Gardner St, Boston, MA'}
{'status': u'Condo For Sale', 'prices': u'345,000', 'lat': -1, 'long': -1, 'address': u'53 Torrey St # 3, Boston, MA'}
{'status': u'Condo For Sale', 'prices': u'675,000', 'lat': -1, 'long': -1, 'address': u'255 Northampton St UNIT 501, Boston, MA'}
{'status': u'House For Sale', 'prices': u'284,999', 'lat': -1, 'long': -1, 'address': u'10 Arbutus St, Dorchester C

### update zip and geo position
this is very important section becaues the first data from the web has no geo position meaning that we cannot use the data with other features. Thus, we use geocoder lib in python and we are able to fill out the lat and long data for each row.

In [24]:
import geocoder

def UpdateZipGeoPos(d):
    total = len(d)
    count = 1.0;
    for i in range(len(d)):
        try:
            address = d[i]['address'] 
            g = geocoder.google(address)
            d[i]['zip'] = g.postal
            if( d[i]['long'] == -1 and d[i]['lat'] == -1):
                d[i]['long'] = g.latlng[1]
                d[i]['lat'] = g.latlng[0]
        except:
            pass
        print "------"
        print (count / total) * 100.0 , " % ..."
        count+=1.0;
    print "done!"
    return d
d = UpdateZipGeoPos(data)




------
3.84615384615  % ...
------
7.69230769231  % ...
------
11.5384615385  % ...
------
15.3846153846  % ...
------
19.2307692308  % ...
------
23.0769230769  % ...
------
26.9230769231  % ...
------
30.7692307692  % ...
------
34.6153846154  % ...
------
38.4615384615  % ...
------
42.3076923077  % ...
------
46.1538461538  % ...
------
50.0  % ...
------
53.8461538462  % ...
------
57.6923076923  % ...
------
61.5384615385  % ...
------
65.3846153846  % ...
------
69.2307692308  % ...
------
73.0769230769  % ...
------
76.9230769231  % ...
------
80.7692307692  % ...
------
84.6153846154  % ...
------
88.4615384615  % ...
------
92.3076923077  % ...
------
96.1538461538  % ...
------
100.0  % ...
done!


In [25]:
print d

[{'status': u'House For Sale', 'zip': u'02125', 'long': -71.049533, 'lat': 42.312072, 'address': u'54 Grampian Way, Dorchester, MA', 'prices': u'899,000'}, {'status': u'Condo For Sale', 'zip': u'02130', 'long': -71.1162116, 'lat': 42.2920089, 'address': u'7-9 Herbertson # 2, Jamaica Plain, MA', 'prices': u'529,000'}, {'status': u'Condo For Sale', 'zip': u'02127', 'long': -71.05343, 'lat': 42.33145, 'address': u'321 Dorchester St # 1, Boston, MA', 'prices': u'849,000'}, {'status': u'House For Sale', 'zip': u'02132', 'long': -71.1544114, 'lat': 42.287886, 'address': u'31 Maple St, Boston, MA', 'prices': u'739,900'}, {'status': u'House For Sale', 'zip': u'02132', 'long': -71.171084, 'lat': 42.277522, 'address': u'129 Gardner St, Boston, MA', 'prices': u'849,000'}, {'status': u'Condo For Sale', 'zip': u'02124', 'long': -71.073962, 'lat': 42.287372, 'address': u'53 Torrey St # 3, Boston, MA', 'prices': u'345,000'}, {'status': u'Condo For Sale', 'zip': u'02118', 'long': -71.0814012, 'lat': 4

In [26]:
print data[0]
for i in d:
    print i['zip']
    print i['long'], i['lat']
    break


{'status': u'House For Sale', 'zip': u'02125', 'long': -71.049533, 'lat': 42.312072, 'address': u'54 Grampian Way, Dorchester, MA', 'prices': u'899,000'}
02125
-71.049533 42.312072


Now, we are able to get deep feature provieded by Zillow with the zip and adress data. 

### data Parsing From Zillow with the zip and address data

In [57]:
from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults, GetUpdatedPropertyDetails 

def GetresultProperty_UpdatedPropertyDetails(YOUR_ZILLOW_ID, address, zipcode):
    YOUR_ZILLOW_API_KEY = 'X1-ZWz1fijun6che3_9ljvt'
    try:
        zillow_id = YOUR_ZILLOW_ID
        zillow_data = ZillowWrapper(YOUR_ZILLOW_API_KEY) 
        updated_property_details_response = zillow_data.get_updated_property_details(zillow_id) 
        result = GetUpdatedPropertyDetails(updated_property_details_response) 
        resultProperty = {}
        resultProperty['zillow_id'] =  result.zillow_id
        resultProperty['home_type'] =  result.home_type
        resultProperty['home_detail_link'] =  result.home_detail_link
        resultProperty['photo_gallery'] =  result.photo_gallery
        resultProperty['latitude'] =  result.latitude
        resultProperty['latitude'] =  result.latitude
        resultProperty['year_built'] =  result.year_built
        resultProperty['property_size'] =  result.property_size
        resultProperty['home_size'] =  result.home_size
        resultProperty['bathrooms'] =  result.bathrooms
        resultProperty['bedrooms'] =  result.bedrooms
        resultProperty['home_info'] =  result.home_info
        resultProperty['year_updated'] =  result.year_updated
        resultProperty['floor_material'] =  result.floor_material
        resultProperty['num_floors'] =  result.num_floors
        resultProperty['basement'] =  result.basement
        resultProperty['roof'] =  result.roof
        resultProperty['view'] =  result.view
        resultProperty['parking_type'] =  result.parking_type
        resultProperty['heating_sources'] =  result.heating_sources
        resultProperty['heating_system'] =  result.heating_system
        resultProperty['rooms'] =  result.rooms
        resultProperty['home_description'] =  result.home_description
        resultProperty['neighborhood'] =  result.neighborhood
        resultProperty['school_district'] =  result.school_district
    except:
        pass
    return resultProperty

def ParsingFromZillow(d):
    YOUR_ZILLOW_API_KEY = 'X1-ZWz1fijun6che3_9ljvt'
    total = len(d)
    count = 0;
    for i in range(len(d)):

        try:
            address = d[i]['address']
            zipcode = d[i]['zip']

            zillow_data = ZillowWrapper(YOUR_ZILLOW_API_KEY)
            deep_search_response = zillow_data.get_deep_search_results(address, zipcode)
            
            #resultProperty = GetUpdatedPropertyDetails(deep_search_response)
            result = GetDeepSearchResults(deep_search_response)

            zillow = {}
            zillow['zillow_id'] = result.zillow_id
            zillow['home_type'] = result.home_type
            zillow['home_detail_link'] = result.home_detail_link
            zillow['graph_data_link'] = result.graph_data_link
            zillow['map_this_home_link'] = result.map_this_home_link
            zillow['latitude'] = result.latitude
            zillow['longitude'] = result.longitude

            zillow['tax_year'] = result.tax_year
            zillow['tax_value'] = result.tax_value
            zillow['year_built'] = result.year_built
            zillow['property_size'] = result.property_size
            zillow['home_size'] = result.home_size
            zillow['bathrooms'] = result.bathrooms
            zillow['bedrooms'] = result.bedrooms
            zillow['last_sold_date'] = result.last_sold_date

            zillow['last_sold_price'] = result.last_sold_price
            zillow['zestimate_amount'] = result.zestimate_amount
            zillow['zestimate_last_updated'] = result.zestimate_last_updated
            zillow['zestimate_value_change'] = result.zestimate_value_change
            zillow['zestimate_valuation_range_high'] = result.zestimate_valuation_range_high
            zillow['zestimate_valuationRange_low'] = result.zestimate_valuationRange_low
            zillow['zestimate_percentile'] = result.zestimate_percentile
            d[i]['DeepSearch'] = zillow
            d[i]["property"] = GetresultProperty_UpdatedPropertyDetails(zillow['zillow_id'] , address, zipcode)
            print i , + "works" 
        except:
            print i , "pass...." 
        print (count / total) * 100.0 , " % ..."
        count+=1.0;
    print "done"
    return d


In [29]:
data = ParsingFromZillow(d)


[{'address': u'54 Grampian Way, Dorchester, MA',
  'lat': 42.312072,
  'long': -71.049533,
  'prices': u'899,000',
  'status': u'House For Sale',
  'zillow': {'bathrooms': '2.0',
   'bedrooms': '5',
   'graph_data_link': 'http://www.zillow.com/homedetails/54-Grampian-Way-Dorchester-MA-02125/59113320_zpid/#charts-and-data',
   'home_detail_link': 'http://www.zillow.com/homedetails/54-Grampian-Way-Dorchester-MA-02125/59113320_zpid/',
   'home_size': '2423',
   'home_type': 'SingleFamily',
   'last_sold_date': '12/11/1998',
   'last_sold_price': '197000',
   'latitude': '42.312059',
   'longitude': '-71.049545',
   'map_this_home_link': 'http://www.zillow.com/homes/59113320_zpid/',
   'property_size': '6534',
   'tax_value': '499300.0',
   'tax_year': '2015',
   'year_built': '1885',
   'zestimate_amount': '769371',
   'zestimate_last_updated': '11/16/2016',
   'zestimate_percentile': '0',
   'zestimate_valuationRange_low': '700128',
   'zestimate_valuation_range_high': '854002',
   'zest

In [17]:
data

{'neighborhood': None, 'bathrooms': '3.0', 'school_district': None, 'heating_sources': 'Gas', 'rooms': None, 'home_description': 'Hot Jeffries Point 2-family for sale! Here is a 2/2 bi-level owners unit that was gutted to the studs a few years ago with a 1/1 first floor apartment that will bring in extra income. There is a finished room in the basement, so why not bi-level that unit too for even more money? Why buy a condo when you can buy a two family for the same price? Please see the video, pictures, floor plans, former income and expenses and improvements to become familiar with the property before coming to view it. The property will be delivered vacant. Cash and as is sales will be highly considered, however a higher sales price will be considered if the price. terms and conditions are acceptable to the sellers. All offers due by Monday at 12:00 and will be answered on Monday evening at 8PM. Property will be delivered vacant. Fast Close for cash as-is will be put to the front of 

In [22]:
data

[{'address': u'54 Grampian Way, Dorchester, MA',
  'lat': 42.312072,
  'long': -71.049533,
  'prices': u'899,000',
  'status': u'House For Sale',
  'zillow': {'bathrooms': '2.0',
   'bedrooms': '5',
   'graph_data_link': 'http://www.zillow.com/homedetails/54-Grampian-Way-Dorchester-MA-02125/59113320_zpid/#charts-and-data',
   'home_detail_link': 'http://www.zillow.com/homedetails/54-Grampian-Way-Dorchester-MA-02125/59113320_zpid/',
   'home_size': '2423',
   'home_type': 'SingleFamily',
   'last_sold_date': '12/11/1998',
   'last_sold_price': '197000',
   'latitude': '42.312059',
   'longitude': '-71.049545',
   'map_this_home_link': 'http://www.zillow.com/homes/59113320_zpid/',
   'property_size': '6534',
   'tax_value': '499300.0',
   'tax_year': '2015',
   'year_built': '1885',
   'zestimate_amount': '769371',
   'zestimate_last_updated': '11/16/2016',
   'zestimate_percentile': '0',
   'zestimate_valuationRange_low': '700128',
   'zestimate_valuation_range_high': '854002',
   'zest

In [17]:
print data

[{'status': u'Condo For Sale', 'zip': u'02127', 'long': -71.05405100000002, 'lat': 42.331281, 'address': u'321 Dorchester St # 1, Boston, MA', 'prices': u'849,000', 'zillow': {'tax_value': None, 'last_sold_price': None, 'property_size': None, 'zestimate_amount': '766086', 'bathrooms': '2.0', 'zestimate_valuation_range_high': '804390', 'tax_year': None, 'zestimate_value_change': None, 'latitude': '42.33145', 'zestimate_percentile': '0', 'bedrooms': '4', 'zestimate_last_updated': '11/13/2016', 'zillow_id': '2096497652', 'last_sold_date': None, 'year_built': None, 'zestimate_valuationRange_low': '727782', 'graph_data_link': None, 'home_size': '1400', 'longitude': '-71.05343', 'map_this_home_link': 'http://www.zillow.com/homes/2096497652_zpid/', 'home_type': 'Condominium', 'home_detail_link': 'http://www.zillow.com/homedetails/321-Dorchester-St-1-Boston-MA-02127/2096497652_zpid/'}}, {'status': u'House For Sale', 'zip': u'02125', 'long': -71.049533, 'lat': 42.312072, 'address': u'54 Grampia

In [30]:
countTotal = 0
countNone = 0
for i in data:
    if (i['zip'] == None):
        countNone += 1
    else:
        countTotal += 1
print 'the number of data is', countTotal
print 'the number of null is', countNone

the number of data is 25
the number of null is 1


Finally, we get the untimate data as a form of Json for web based data visualization and CSV for training models.

In [31]:
def SaveJson(path, data): # "source\\K-town_streetviews.json"
    with open(path, 'w') as f:
#         f.write(json.dumps(data, indent=4, ensure_ascii=False, encoding="latin1"))
        f.write(json.dumps(data, indent=4, encoding="latin1"))
        f.close()
def GetPandasFromDict(d):
    return pd.DataFrame.from_dict(d)
def GetDictFromPandas(d):
    return pd.DataFrame.to_dict(d)

data = d
thePd = GetPandasFromDict(data)
theDict = GetDictFromPandas(thePd)

# SaveJson("HousingPriceFromWebTestWithProperties.json",data)

print "done"

done


### /////////////////////////////////////////////////////////////////// done 

### (this is for the additional data)

In [58]:
def OpenJson(path):
    json_file = open(path , 'r')
    json_str = json_file.read()
    json_data = json.loads(json_str)
    return json_data
def SaveJson(path, data): 
    with open(path, 'w') as f:
        f.write(json.dumps(data, indent=4,  encoding="latin1"))
        f.close()
    print "done for save"



In [59]:
d = OpenJson("HousingPriceFromWeb.json")
count = 0;
total = len(d);
for i in range(len(d)):
    try:
        theZip = d[i]['zip']
        theAddress = d[i]['address']
        theID = d[i]['zillow']['zillow_id']
        d[i]['property'] =  GetresultProperty_UpdatedPropertyDetails(theID, theAddress, theZip)
    except:
        pass
    print (count / total) * 100.0 , " % ..."  
    count += 1

In [61]:
print d

[{u'status': u'Condo For Sale', u'zip': u'02127', u'long': -71.05405100000002, u'prices': u'849,000', u'address': u'321 Dorchester St # 1, Boston, MA', u'lat': 42.331281, 'property': {'neighborhood': None, 'bathrooms': '2.0', 'school_district': None, 'heating_sources': None, 'rooms': None, 'home_description': '321 Dorchester Street is a boutique luxury development in the heart of South Boston. These floor-through condominiums feature high end kitchens with Carrera marble tops, Samsung appliances and ample storage. Each residence offers 4 bedrooms, 2 bathrooms and access to the common roof deck with a panoramic Boston skyline views.', 'property_size': None, 'num_floors': None, 'heating_system': None, 'photo_gallery': 'http://www.zillow.com/homedetails/321-Dorchester-St-1-Boston-MA-02127/2096497652_zpid/#image=lightbox%3Dtrue', 'floor_material': None, 'year_updated': None, 'bedrooms': '4', 'latitude': '42.33145', 'zillow_id': '2096497652', 'basement': None, 'year_built': None, 'home_size

In [63]:
countProperty = 0;
countTotal = 0
for i in d:
    try:
        a = i['property']
        countProperty += 1 
    except:
        pass
    countTotal +=1
print "the total number of data is" , countTotal, " and the number of property is ", countProperty


the total number of data is 988  and the number of property is  102


In [ ]:
SaveJson("HousingPriceFromWebTestWithProperties.json",d)